In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
descriptions = pd.read_excel('LCDataDictionary.xlsx')
descriptions.rename(columns={'LoanStatNew': 'name', 'Description': 'description'}, inplace=True)
descriptions.sample(5)

,name,description
127,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...
33,installment,The monthly payment owed by the borrower if th...
70,num_tl_30dpd,Number of accounts currently 30 days past due ...
24,funded_amnt_inv,The total amount committed by investors for th...
90,revol_bal,Total credit revolving balance


In [ ]:
# download from https://www.dropbox.com/scl/fi/rzqaawjqwt4qe3rmnnxiw/loan_data_2007_2014.csv?rlkey=a5y6ojznit1ozu8fwt0m7w11w&dl=0

df = pd.read_csv('loan_data_2007_2014.csv')
print(df.shape)
df.sample(5).T

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3


In [ ]:
def months_from_date(ser, reference_date = '2024-01-01', time_format='%b-%y'):
    return (
            pd.to_datetime(reference_date).year - pd.to_datetime(ser, format=time_format).dt.year
        ) * 12 + (
            pd.to_datetime(reference_date).month - pd.to_datetime(ser, format=time_format).dt.month
        ).astype('Float64')

def dummy_encode_df(df_, category_columns = [
        "grade",
        "sub_grade",
        "home_ownership",
        "verification_status",
        "loan_status",
        "purpose",
        "addr_state",
        "initial_list_status",
    ]):

    return pd.concat([df_[category_columns], pd.get_dummies(
        df_,
        columns=category_columns,
        drop_first=True
        )], axis=1)

def zfill_df(df_, columns_to_zfill = [
        "mths_since_earliest_cr_line",
        "acc_now_delinq",
        "total_acc",
        "pub_rec",
        "open_acc",
        "inq_last_6mths",
        "delinq_2yrs",
        "emp_length_int",
    ]):

    return df_.assign(**{
        k:df_.loc[:,k].fillna(0) for k in columns_to_zfill
    })

In [255]:
loan_data = (df
             .iloc[:,1:]
             .set_index('id')
             .assign(
                 term_num=lambda df_: df_.term.str.split(expand=True).iloc[:,0],
                 emp_length_int=lambda df_: df_
                                        .emp_length
                                        .replace({
                                            "< 1 year": "0.5", '10+ years': "11"
                                         })
                                        .str.extract(r'(\d+\.?\d*)')
                                        .astype('Float64'),
                mths_since_earliest_cr_line=lambda df_: months_from_date(df_.earliest_cr_line),
                mths_since_issue_d=lambda df_: months_from_date(df_.issue_d),
                total_rev_hi_lim=lambda df_: df_.total_rev_hi_lim.fillna(df_.funded_amnt),
                annual_inc=lambda df_: df_.annual_inc.fillna(df_.annual_inc.mean()),
                good_bad=lambda df_: np.where(df_.loan_status.isin(['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'Late (31-120 days)']),0,1)
             )
             .query("mths_since_earliest_cr_line > 0")
             .pipe(dummy_encode_df)
             .pipe(zfill_df)
)
loan_data

,grade,sub_grade,home_ownership,verification_status,loan_status,purpose,addr_state,initial_list_status,member_id,loan_amnt,...,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w
id,,,,,,,,,,,,,,,,,,,,,
1077501,B,B2,RENT,Verified,Fully Paid,credit_card,AZ,f,1296599,5000,...,False,False,False,False,False,False,False,False,False,False
1077430,C,C4,RENT,Source Verified,Charged Off,car,GA,f,1314167,2500,...,False,False,False,False,False,False,False,False,False,False
1077175,C,C5,RENT,Not Verified,Fully Paid,small_business,IL,f,1313524,2400,...,False,False,False,False,False,False,False,False,False,False
1076863,C,C1,RENT,Source Verified,Fully Paid,other,CA,f,1277178,10000,...,False,False,False,False,False,False,False,False,False,False
1075358,B,B5,RENT,Source Verified,Current,other,OR,f,1311748,3000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598660,C,C2,MORTGAGE,Source Verified,Current,debt_consolidation,TX,w,1440975,18400,...,False,True,False,False,False,False,False,False,False,True
9684700,D,D5,MORTGAGE,Verified,Charged Off,debt_consolidation,TN,f,11536848,22000,...,True,False,False,False,False,False,False,False,False,False
9584776,D,D1,MORTGAGE,Verified,Current,debt_consolidation,OH,f,11436914,20700,...,False,False,False,False,False,False,False,False,False,False


In [256]:
x_train, x_test, y_train, y_test = train_test_split(
    loan_data.drop('good_bad', axis=1),
    loan_data.good_bad,
    test_size=.2,
    random_state=42
    )

In [257]:
x_train_with_target = pd.concat([x_train, y_train], axis=1)
x_train_with_target.sample(5)

,grade,sub_grade,home_ownership,verification_status,loan_status,purpose,addr_state,initial_list_status,member_id,loan_amnt,...,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w,good_bad
id,,,,,,,,,,,,,,,,,,,,,
1080572,C,C5,RENT,Not Verified,Fully Paid,debt_consolidation,MO,f,1317288,2200,...,False,False,False,False,False,False,False,False,False,1
32389696,D,D1,MORTGAGE,Verified,Late (16-30 days),debt_consolidation,NC,w,35002989,12000,...,False,False,False,False,False,False,False,False,True,1
7062771,B,B4,RENT,Verified,Current,debt_consolidation,CA,f,8724349,24000,...,False,False,False,False,False,False,False,False,False,1
10170443,B,B2,MORTGAGE,Not Verified,Current,debt_consolidation,DC,f,12021974,7000,...,False,False,False,False,False,False,False,False,False,1
3917737,C,C4,OWN,Verified,Charged Off,debt_consolidation,VA,w,5010975,11100,...,False,False,True,False,False,False,False,False,True,0


In [270]:
feature = 'grade'
target = 'good_bad'

woe_metrics = (x_train_with_target.groupby(feature, observed=True)[target]
 .agg(prop_good='mean',n_obs='count')
 .assign(
     prop_n_obs=lambda df_: df_.n_obs.div(df_.n_obs.sum()),
     n_good=lambda df_: df_.prop_good * df_.n_obs,
     n_bad=lambda df_: (1-df_.prop_good) * df_.n_obs,
     prop_n_good=lambda df_: df_.n_good.div(df_.n_good.sum()),
     prop_n_bad=lambda df_: df_.n_bad.div(df_.n_bad.sum()),
     WoE=lambda df_: np.log(df_.prop_n_good.div(df_.prop_n_bad)))
 .sort_values(by='WoE', ascending=False)
)

print((woe_metrics.prop_n_good - woe_metrics.prop_n_bad).mul(woe_metrics.WoE).sum())

woe_metrics

0.2958816109036884


,prop_good,n_obs,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WoE
grade,,,,,,,,
A,0.961505,59592,0.160164,57298.0,2294.0,0.172951,0.056263,1.122977
B,0.921852,109126,0.293295,100598.0,8528.0,0.303650,0.209158,0.372785
C,0.885029,99956,0.268649,88464.0,11492.0,0.267024,0.281853,-0.054048
D,0.844769,61489,0.165262,51944.0,9545.0,0.156790,0.234101,-0.400843
E,0.805162,28634,0.076959,23055.0,5579.0,0.069590,0.136831,-0.676119
F,0.756800,10588,0.028457,8013.0,2575.0,0.024187,0.063155,-0.959776
G,0.716841,2684,0.007214,1924.0,760.0,0.005807,0.018640,-1.166149
